# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### Importing the dataset

In [2]:
dataset = pd.read_excel('Peak_Incident_Pressure.xlsx')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Charge_mass        576 non-null    float64
 1   Standoff_distance  576 non-null    int64  
 2   Peak_Pressure      576 non-null    float64
 3   Peak_time          576 non-null    float64
dtypes: float64(3), int64(1)
memory usage: 18.1 KB


In [3]:
y = dataset['Peak_Pressure']
X = dataset.drop(['Peak_time','Peak_Pressure'], axis=1)
print(X.shape, y.shape)

(576, 2) (576,)


In [4]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

### Splitting the dataset into the Training set and Test set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

### Hyperparameter tuning 1 - layers, neurons, activation function

In [6]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor as KR
import math
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [7]:
from tensorflow.keras.optimizers import Adam
def create_model(n_layers, first_layer_nodes, last_layer_nodes, activation_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(units = first_layer_nodes,  input_shape=(X_train.shape[1],), activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation='linear'))
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer = opt, loss = 'mean_squared_error', metrics = ['mae'])
    return model

##Wrap model into scikit-learn
model = KR(build_fn=create_model, epochs = 500, batch_size = 50)

C:\Users\z5379606\AppData\Local\Temp\ipykernel_33788\2168085556.py:18: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KR(build_fn=create_model, epochs = 500, batch_size = 50)


In [8]:
activation_funcs = ['relu', 'softplus', 'leaky_relu'] 
param_grid = dict(n_layers=[2,3],
                  first_layer_nodes = [70, 75, 80],
                  last_layer_nodes = [35, 40, 45],
                  activation_func = activation_funcs)
grid = GridSearchCV(estimator = model, param_grid = param_grid,n_jobs=-1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/500
9/9 [==============================] - 1s 5ms/step - loss: 2171091943424.0000 - mae: 506917.3750
Epoch 2/500
9/9 [==============================] - 0s 1ms/step - loss: 2171085651968.0000 - mae: 506909.8438
Epoch 3/500
9/9 [==============================] - 0s 1ms/step - loss: 2171078049792.0000 - mae: 506901.4375
Epoch 4/500
9/9 [==============================] - 0s 2ms/step - loss: 2171068219392.0000 - mae: 506890.0312
Epoch 5/500
9/9 [==============================] - 0s 2ms/step - loss: 2171054325760.0000 - mae: 506874.2188
Epoch 6/500
9/9 [==============================] - 0s 2ms/step - loss: 2171036368896.0000 - mae: 506851.2500
Epoch 7/500
9/9 [==============================] - 0s 1ms/step - loss: 2171010547712.0000 - mae: 506820.2188
Epoch 8/500
9/9 [==============================] - 0s 2ms/step - loss: 2170978435072.0000 - mae: 506779.1250
Epoch 9/500
9/9 [==============================] - 0s 1ms/step - loss: 2170937671680.0000 - mae: 506723.8438
Epoch 10/500
9/9 [=

### Hyperparameter tuning - batch size, epoch, optimizer, learning rate

In [9]:
# Function to create model, required for KerasRegressor
def create_model(optimizer):
 # create model
 model = Sequential()
 model.add(Dense(units=75, input_shape=(X_train.shape[1],), activation='softplus'))
 model.add(Dense(units=55, activation='softplus'))
 model.add(Dense(units=35, activation='softplus'))
 model.add(Dense(units=1, activation='linear'))
 # Compile model
 model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['mae'])
 return model

In [10]:
from scikeras.wrappers import KerasRegressor
# create model
model = KerasRegressor(model=create_model)

# define the grid search parameters
batch_size = [30,40,50]
epochs = [300, 400, 500]
optimizer = ['RMSprop', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1]

# gridsearch
param_grid = dict(batch_size=batch_size, epochs=epochs, model__optimizer=optimizer, optimizer__learning_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/500
11/11 [==============================] - 1s 2ms/step - loss: 0.1311 - mae: 0.2609
Epoch 2/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0240 - mae: 0.1230
Epoch 3/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0061 - mae: 0.0646
Epoch 4/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0037 - mae: 0.0462
Epoch 5/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0026 - mae: 0.0384
Epoch 6/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0014 - mae: 0.0310
Epoch 7/500
11/11 [==============================] - 0s 2ms/step - loss: 7.9405e-04 - mae: 0.0221
Epoch 8/500
11/11 [==============================] - 0s 2ms/step - loss: 6.3019e-04 - mae: 0.0191
Epoch 9/500
11/11 [==============================] - 0s 2ms/step - loss: 3.6557e-04 - mae: 0.0149
Epoch 10/500
11/11 [==============================] - 0s 2ms/step - loss: 3.1167e-04 - mae: 0.0141
Epoch 11/500
11/11 [=======================